In [ ]:
import pandas as pd
import litellm
import os
import re
from google import genai
from google.genai import types
from google.genai.errors import APIError
from pathlib import Path
import json
import boto3
from botocore.exceptions import NoCredentialsError
from dotenv import load_dotenv

In [ ]:
# API_KEY = insert your own
# client = genai.Client(api_key=API_KEY)
load_dotenv()
api_key = os.getenv("API_KEY")
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
BUCKET_NAME = "linqia-video-analyser-bucket"
client = genai.Client(api_key=api_key)


In [ ]:
NEW_PROMPT_TEXT = """
You are an AI video analysis system. Follow all rules strictly. Accuracy, minimal labeling, and confidence-based decisions are mandatory.

────────────────────────────────────────
NO-HALLUCINATION RULES
────────────────────────────────────────
- NEVER guess, assume, or infer beyond confirmed visual or audible evidence.
- If content is unclear, ambiguous, partially visible, or conflicting → return “", or exclude the label entirely. If there is any doubt, the label does NOT apply.
- Do NOT overlabel. You are encouraged to return the fewest possible accurate labels. Excessive or unnecessary labelling is considered a failure.
- If multiple interpretations are possible, choose the LEAST specific valid interpretation.
- Assume nothing. This video stands alone.
- Omission is always preferable to false inclusion. Inaction > Wrong Action. However, do NOT omit observable events, interactions, settings, or objects simply to keep the summary short.
- Use objective, factual language only. Avoid emotional, motivational or speculative phrasing.
- Segment summaries if scenes, speakers, or actions change.
- Prioritize completeness while maintaining clarity and avoiding unnecessary repetition.

────────────────────────────────────────
CONTENT THEME RULES
────────────────────────────────────────
- Select:
  • 1 Primary theme (required if any apply)
  • Up to 4 Secondary themes (optional)
- Do not exceed 5 total content_theme labels unless the video clearly contains multiple unrelated segments.
- Themes must reflect the PRIMARY focus of the video.
- Do not label based on incidental objects or genre familiarity.
- Broad categories like “Lifestyle” or “Education” require strong confirmation from visuals or transcript.

Lifestyle is NEVER a default.
Assign “Lifestyle” ONLY if the core focus is daily life, routines, or intentional living.
If another category fits better, use that instead.

────────────────────────────────────────
CONTENT STYLE RULES
────────────────────────────────────────
- Content style is determined primarily from the transcript and narrative structure and not just visual location changes alone. 
- Visual cues may support a style but must NOT override transcript structure. 

Vlogs (STRICT):
- Assign ONLY if the video is diary-style and centered on the creator’s
  personal experiences or reflections.
- Do NOT assign to list-based, informational, or guide-style videos.

Tutorial:
- Label as “Tutorial” ONLY IF:
  a) The transcript includes explicit instructional language:
     ("how to", "step one", "let me show you", etc.), OR  
  b) The video clearly demonstrates an ordered teaching sequence.
- Demonstration alone ≠ tutorial.

Transitions:
- Only if clear editing cuts/fades/jump cuts are visible.
- Camera movement alone ≠ transitions.
- If unclear → Return empty.

Montages / Compilations:
- Multiple similar clips shown briefly with repeated cuts and no single narrative.

OOTD:
- Assign “Outfit of the Day” if outfit showcasing/posing/styling occurs AND/OR “OOTD” / “outfit of the day” is spoken or shown.

“Day-in-the-Life” ONLY IF ALL of the following are true:
1. The video follows a chronological progression of the creator’s day
   (e.g., morning → afternoon → evening).
2. The creator explicitly frames the content as their own day or routine
   (e.g., “a day in my life”, “come with me today”, “here’s my day”).
3. The primary focus is the creator’s lived experience, not recommendations,
   tips, rankings, or demonstrations.

LISTICLE OVERRIDE (CRITICAL):
- If the transcript presents numbered, ordered, or enumerated items
  (e.g., “five things to do”, “top places”, “next up”, “last but not least”),
  classify the video as “Listicles”.
- When this applies, DO NOT assign Vlogs, Day-in-the-Life, or Montages.

────────────────────────────────────────
CREATOR PRESENCE RULES
────────────────────────────────────────
- Choose ONLY from visible evidence.
- Do not select logically conflicting categories.
- “FACE ONLY” → select ONLY if no part of the body beyond face/neck is visible at ANY time.
- “FULL BODY” → select if head, torso, and legs are visible at the same time in at least one frame.
- “UPPER BODY” → select if torso and arms are visible but legs are not.
- "LOWER BODY" → select if legs and lower torso are visible but upper body/face are not.

────────────────────────────────────────
ON-SCREEN TEXT NORMALIZATION (CAPTIONS RULE):
────────────────────────────────────────
- If on-screen text substantially matches or paraphrases the spoken transcript,
  classify it ONLY as “Closed Captioning” or “Subtitles”.
- Do NOT additionally label quotes, titles, headlines, or animated text
  when the text is merely duplicating spoken audio.
- Only assign other on-screen text labels if the text adds NEW information
  not present in the transcript.

────────────────────────────────────────
BRAND SAFETY RULES
────────────────────────────────────────
Evaluate risk across ALL modalities:
a) AUDIO (spoken words & lyrics)  
b) VISUALS (graphic/explicit imagery)  
c) ON-SCREEN TEXT (if provided)

Language & Lyrics:
- Treat background music lyrics as equal to spoken dialogue.
- If profane, explicit, or offensive lyrics are clearly audible → flag appropriately.
- If lyrics are unintelligible → return "Unknown".

Severity (mutually exclusive):
1. Strong language
2. Mild language
If strong applies, do NOT include mild.
If unsure, choose the LESS severe option.

Visual/Text Risk:
- Flag only if clearly graphic or explicit (e.g. blood, wounds, nudity, profanity).
- Do NOT reduce severity based on medical or educational context.
- Suggestive but unclear → “Unknown”.

PROFANITY SEVERITY (STRICT)
- Assign “Strong language” if ANY explicit strong-profane terms are spoken,
  regardless of tone or context.
  Examples include: "fuck", "fucking", "motherfucker", "cunt",
  explicit sexual terms, slurs, or hate speech.
- Assign “Mild language” ONLY for light, non-explicit profanity
  (e.g., “damn”, “hell”) AND ONLY if no strong-profane terms are present.
- Severity labels are mutually exclusive.
  If strong applies, omit mild.
Exception:
- The “choose the less severe option” rule does NOT apply
  when strong-profane terms are present.

────────────────────────────────────────
BRAND / LOGO DETECTION
────────────────────────────────────────
Identify a brand ONLY if:
- A logo is clearly readable on-screen AND represents a product, service, or company
  that is part of the video content itself OR
- The brand name is explicitly spoken.

PLATFORM EXCLUSION (NON-NEGOTIABLE):
- Platform watermarks, banners, UI elements, or hosting indicators
  (e.g., TikTok, Instagram, YouTube, Reels, Shorts)
  are NOT brands and MUST NOT be included.

FINAL BRAND CLEANUP (REQUIRED):
- After all brand detection rules are applied, REMOVE any platform names
  (e.g., TikTok, Instagram, YouTube, Reels, Shorts).
- If no valid brands remain, brand_names MUST be an empty array [].
- Do NOT substitute platform names or placeholders.

────────────────────────────────────────
TRANSCRIPTION RULES
────────────────────────────────────────
- Transcribe ONLY spoken foreground audio.
- Do NOT include on-screen text unless spoken.
- Do NOT describe visuals.
- Remove filler words and false starts.
- Return a single-line plain-text transcript with clean punctuation.
- Do NOT transcribe background music lyrics unless:
  a) The lyrics are clearly foregrounded and intended to be heard as speech, OR
  b) The speaker is performing/singing as the primary content.

────────────────────────────────────────
TASKS
────────────────────────────────────────
A) SUMMARY  
Provide a concise, strictly factual summary using only confirmed audio/visual evidence. Do NOT include inferred motivations, emotions, or assumptions.

B) VIDEO CLASSIFICATION  
Select labels ONLY from the lists below.  
If no confident match → “No category”.

────────────────────────────────────────
LABEL SETS (MUST USE EXACTLY)
────────────────────────────────────────

1) CONTENT THEME (choose MULTIPLE)
[Recipes, Cooking, Baking, Food Reviews, Restaurant Tours, Pets, Dog Training, Cat Care, Exotic Animals, Travel, Budget Travel, Luxury Travel, Adventure Travel, Family, Parenting, Single Parenting, Large Family Life, Cars, Car Reviews, Auto Repair, Classic Cars, Fashion, Streetwear, High Fashion, Thrifting, Beauty, Skincare, Makeup Tutorials, Hair Styling, Fitness, Weightlifting, Yoga, Pilates, CrossFit, Gymnastics, Technology, Gadget Reviews, Coding Tutorials, Tech News, Gaming, Game Reviews, Let's Plays, Esports, Music, Instrument Tutorials, Music Production, Song Covers, DIY, Crafts, Home Improvement, Upcycling, Arts and Crafts, Education, Language Learning, Science Experiments, History, Humor, Stand-up Comedy, Pranks, Sketch Comedy, Lifestyle, Luxury Living, Van Life, Personal Finance, Budgeting Tips, Investing Basics, Cryptocurrency, Mental Health, Meditation, Therapy Insights, Self-Care Tips, Gardening, Urban Gardening, Permaculture, Plant Care, Sports, Sports Analysis, Athlete Interviews, Training Tips, Books, Book Reviews, Author Interviews, Reading Challenges, Art, Painting Tutorials, Digital Art, Photography, Camera Reviews, Photo Editing Tutorials, Photography Tips, Environmental, Sustainability Tips, Climate Change Awareness, Zero Waste Living, Politics, Political Analysis, Activism, Civic Education, Spirituality, Meditation Guides, Religious Teachings, New Age Practices, Business, Entrepreneurship, Marketing Tips, Small Business Advice, Science, Space Exploration, Biology Facts, Chemistry Experiments, Film/TV, Movie Reviews, TV Show Recaps, Behind-the-Scenes, Fashion, Outfit Ideas, Fashion History, Sustainable Fashion, Oddly Satisfying, Sensory Video, Debating, Social Commentary, Philosophy, Study, Academics, Exam Preparation, Research, Student Life, Campus Tours, Interview Tips, Racing, Religion, Extreme Sports, Wildlife, Motivation, Famous Quotes, Walking Tour, Health & Wellbeing, Medical, Career, Aviation, Geography, Investments, Property, Celebrities, Entertainment News, Back to School, Couples, Crime, Dancing, KPOP, Korean Dramas, Pregnancy, Weddings, Nutrition]

2) CONTENT STYLE (choose MULTIPLE)
[ASMR, Skits, Transitions, Graphics-heavy, Vlogs, Day-in-the-Life, Tutorials, Reviews, Unboxing, Challenges, Q&A, Interviews, Reaction Videos, Compilations, Time-lapse, Slow Motion, Stop Motion, Live Streaming, Storytelling, Podcasts, Listicles, How-To Guides, Before and After, Pranks, Social Experiments, Parodies, Montages, Behind-the-Scenes, Hauls, Lookbooks, Product Demos, Taste Tests, Mukbang, Room Tours, Workout Routines, Outfit of the Day, Get Ready With Me, What I Eatil in a Day, Morning Routines, Night Routines, Transformation Videos, Expectation vs. Reality, Duexts/Collaborations, Rants, Storytime, Ambient/Background Videos, Educational Animations, Whiteboard Explainers, Green Screen Effects, Point-of-View (POV), Lip Sync, Choreography, Talent Showcases, Response Videos, Text-based, Talking-Head, Ragebait, Philosophical, Motivational Content, Study With Me, Study Tips, Tips and Tricks, Advice, Quiz, Travel Itineraries, Case Studies, Street Interview, What's In My Bag, Giveaways, Product Reviews, Service Reviews, Success Stories, Drone Footage, Cinematic]

3) CREATOR PRESENCE (choose MULTIPLE)
[Hands-only, Creator-centric, Silhouettes, No visible people, Group content, Face only, Full body, Upper Body, Lower Body, Voice-only, Animated, Cameo appearances, Behind the camera]

4) ON-SCREEN TEXT / GRAPHICS (choose MULTIPLE)
["POV", Closed Captioning, Brand Logos, FTC Disclosures (#ad/#brandpartner), Subtitles, Emojis, Infographics, Statistics, Quotes, Timestamps, Pricing Information, Product Names, Usernames/Handles, Clickable Links, Countdown Timers, Call-to-Action Buttons, Animated Text, Lyric Displays, Screen Overlays, Lower Thirds, Pop-up Bubbles, Tutorial Steps, Recipe Ingredients, Titles/Headlines, End Screens, Watermarks, Location Tags, Age Restrictions, Content Warnings, Fact Boxes, Scoreboard/Leaderboard]

5) KEY VIDEO ELEMENTS (choose MULTIPLE)
[Creator, Product, Pet, Child, Text/Graphics, Music/Sound, Special Effects, Props, Food, Vehicles, Technology, Nature, Urban Environment, Artwork, Sports Equipment, Clothing/Fashion Items, Books, Musical Instruments, Fitness Equipment, Beauty Products, Home Decor, DIY Projects, Collectibles, Celebrity Guests, Expert Interviews, Audience Participation, Screen Recordings, Archival Footage]

6) BRAND SAFETY CATEGORIES (choose MULTIPLE)
[Family-friendly, Mild language, Strong language, Sexual content, Violence, Drug use, Alcohol consumption, Revealing clothing, Controversial topics, Political content, Religious content, Graphic medical content, Dangerous stunts, Potential copyright issues, Unverified health claims, Financial advice, Sensitive social issues, Depictions of gambling, Weapons, Extreme sports, Conspiracy theories, Potential misinformation, Satire/parody]

7) BRAND TYPE (choose MULTIPLE)
[No brands featured, Multiple brands featured, Exclusive brand partnership, Competitor brands shown, Small/local brands, Luxury brands, Tech brands, Fashion brands, Beauty brands, Food and beverage brands, Automotive brands, Travel/hospitality brands, Fitness/health brands, Home goods brands, Entertainment brands, Financial services brands, Educational brands, Pet care brands, Sustainability-focused brands, Sports equipment brands, Gaming brands, Subscription services, Direct-to-consumer brands, Retail chains]

8) BRAND NAMES
- Brand_names MUST be an array of strings.
- If no valid brands are present after applying ALL exclusions,
  return an EMPTY ARRAY: [].
- An empty array is the correct and final output when no brands apply.
- Do NOT add platform names (e.g., TikTok) to avoid returning an empty array.



────────────────────────────────────────
OUTPUT — STRICT JSON ONLY
────────────────────────────────────────
{
  "summary": "",
  "content_theme": "",
  "content_style": "",
  "creator_presence": "",
  "on_screen_text": [],
  "key_video_elements": [],
  "brand_safety": [],
  "brand_type": [],
  "brand_names": [],
  "transcript": ""
}

Return ONLY this JSON. No explanations.

"""

In [ ]:
def analyze_video_url(video_url: str, prompt_text: str = NEW_PROMPT_TEXT) -> str:
    """
    Analyzes a single video URL using litellm and the Gemini model.

    """
    if not video_url or not isinstance(video_url, str) or not video_url.startswith('http'):
        print(f"Skipping invalid URL: {video_url}")
        return "Error: Invalid or empty URL"

    print(f"Analyzing video: {video_url}...")
    
    # This is the message structure for multimodal input (text + video)
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt_text
                },
                {
                    "type": "image_url",  # This key is used for video as well
                    "image_url": {
                        "url": video_url
                    }
                }
            ]
        }
    ]
    
    try:
        response = litellm.completion(
            model = "gemini/gemini-2.5-flash-lite",
            messages=messages,
            api_key=api_key,
            timeout=300 # 5-minute timeout for potentially long videos
        )
        # Extract the text content from the response
        content = response.choices[0].message.content
        print("Analysis complete.")
        return content
    except Exception as e:
        print(f"Error analyzing video {video_url}: {e}")
        return f"Error: {e}"

In [ ]:
def extract_json(response_text: str) -> dict:
    """
    Attempts to extract JSON from response text that might have markdown code blocks or extra text.
    Returns a Python dictionary.
    """
    
    # Try to parse & read as-is (pure JSON) first. Great if it works, else try other method.
    try:
        return json.loads(response_text)
    except json.JSONDecodeError:
        pass
    
    # Try to extract JSON from markdown code blocks
    # Look for the pattern ```json {...}``` & extract parts b/w the code block markers
    json_pattern = r'```(?:json)?\s*(\{.*?\})\s*```'
    matches = re.findall(json_pattern, response_text, re.DOTALL) 
    
    if matches:
        try:
            return json.loads(matches[0]) # 
        except json.JSONDecodeError:
            pass
    
    # Try to find JSON object in the text (look for { ... })
    try:
        start = response_text.find('{')
        end = response_text.rfind('}') + 1
        if start != -1 and end > start:
            json_str = response_text[start:end]
            return json.loads(json_str)
    except json.JSONDecodeError:
        pass
    
    # If all else fails, return None
    return None


In [ ]:
def get_public_s3_urls(bucket_name: str, AWS_ACCESS_KEY: str, AWS_SECRET_KEY: str, region: str = "us-east-2"):
    """
    Connects to S3 buckets, finds and creates public URLs for all video files.
    """
    s3_client = boto3.client('s3',
        aws_access_key_id=AWS_ACCESS_KEY, # UPDATE THIS VALUE
        aws_secret_access_key=AWS_SECRET_KEY, # UPDATE THIS VALUE
        region_name=region # UPDATE THIS VALUE if different region
    )
    
    video_urls = {}
    
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        
        if 'Contents' not in response: # Checks if there are any files in the bucket
            print("No files found in bucket")
            return {}
        
        for obj in response['Contents']:
            file_name = obj['Key']
            
            if any(file_name.endswith(ext) for ext in ['.mp4', '.mov', '.avi', '.mkv']):
                url = f"https://{bucket_name}.s3.{region}.amazonaws.com/{file_name}"
                # Replace spaces and special characters with URL-safe codes so web browsers can understand them
                url_encoded = url.replace(' ', '%20').replace('&', '%26')
                
                # "Videos for Analysis/Auto Repair.mp4" -> ["Videos for Analysis", "Auto Repair.mp4"]
                # Take last part "Auto Repair.mp4"
                clean_name = file_name.split('/')[-1] 
                video_urls[clean_name] = url_encoded
                print(f"{clean_name}") # Shows progress as each video is found
        
        return video_urls
        
    except Exception as e:
        print(f"Error: {e}")
        return None


In [ ]:
def convert_analyses_to_csv(results: list, output_csv: str = "video_analyses_summary.csv"):
    """
    Helper function to convert JSON analysis results to CSV.
    """
    csv_rows = []
    
    for result in results:
        # Attempt to extract JSON from the response text
        analysis_json = extract_json(result['analysis'])
        
        if analysis_json:
            row = {
                'video_name': result['video_name'],
                'video_url': result.get('url', 'N/A'),
                'summary': analysis_json.get('summary', ''),
                'content_theme': analysis_json.get('content_theme', ''),
                'content_style': analysis_json.get('content_style', ''),
                'creator_presence': analysis_json.get('creator_presence', ''),
                'on_screen_text': ', '.join(analysis_json.get('on_screen_text', [])) if isinstance(analysis_json.get('on_screen_text'), list) else analysis_json.get('on_screen_text', ''),
                'key_video_elements': ', '.join(analysis_json.get('key_video_elements', [])) if isinstance(analysis_json.get('key_video_elements'), list) else analysis_json.get('key_video_elements', ''),
                'brand_safety': ', '.join(analysis_json.get('brand_safety', [])) if isinstance(analysis_json.get('brand_safety'), list) else analysis_json.get('brand_safety', ''),
                'brand_type': analysis_json.get('brand_type', ''),
                'brand_names': ', '.join(analysis_json.get('brand_names', [])) if isinstance(analysis_json.get('brand_names'), list) else analysis_json.get('brand_names', ''),
                'transcript': analysis_json.get('transcript', '')
            }
            csv_rows.append(row)
        else:
            # If JSON extraction failed, save error info
            csv_rows.append({
                'video_name': result['video_name'],
                'video_url': result.get('url', 'N/A'),
                'summary': 'ERROR: Could not parse response',
                'content_theme': '',
                'content_style': '',
                'creator_presence': '',
                'on_screen_text': '',
                'key_video_elements': '',
                'brand_safety': '',
                'brand_type': '',
                'brand_names': '',
                'transcript': result['analysis'][:500]  # First 500 chars of raw response
            })
    
    df = pd.DataFrame(csv_rows)
    df.to_csv(output_csv, index=False, encoding='utf-8')
    
    print(f"\n✓ Saved CSV with {len(csv_rows)} videos to: {output_csv}")
    
    return df

In [ ]:
def analyze_s3_videos_with_csv(video_urls: dict, output_folder: str = "testing_18"):
    """
    Analyze videos from S3 URLs and save it as JSON + CSV outputs.
    """
    os.makedirs(output_folder, exist_ok=True)
    results = []
    
    for i, (video_name, url) in enumerate(video_urls.items(), 1):
        
        analysis = analyze_video_url(url, NEW_PROMPT_TEXT)
        
        # Save individual JSON
        output_filename = f"{Path(video_name).stem}_analysis.json"
        output_path = os.path.join(output_folder, output_filename)
        
        # Try to extract and save clean JSON
        analysis_json = extract_json(analysis)
        
        # If JSON extraction succeeds, save as JSON, else save raw text
        if analysis_json:
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(analysis_json, f, indent=2, ensure_ascii=False)
        else:
            output_path = output_path.replace('.json', '.txt')
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(analysis)
    
        results.append({
            'video_name': video_name,
            'url': url,
            'output_file': output_path,
            'analysis': analysis
        })
        
        print(results)

    
    # Convert to CSV
    csv_output = os.path.join(output_folder, "video_analysis_summary_18.csv")
    df = convert_analyses_to_csv(results, csv_output)
    
    print(f"\n📊 Output: {csv_output}")
    
    return results, df

In [ ]:
# Get video URLs from S3
video_urls = get_public_s3_urls(BUCKET_NAME, AWS_ACCESS_KEY, AWS_SECRET_KEY, region="us-east-2")

if video_urls:
    results, analysis_df = analyze_s3_videos_with_csv(video_urls)
    
    # # Display first 3 summaries to verify
    # print(analysis_df[['video_name', 'summary']].head(3))

In [ ]:
# # Old Prompt Text Output using Video URL
# video_url = "https://resonate-media-cdn.internal.linqia.com/influencer-content-converted-videos/2378c553fc1a1edab4ae5c3786903c92c5f4b8d9662f0e5e4e695a6c26c114a1.mp4"
# output = analyze_video_url(video_url, OLD_PROMPT_TEXT)
# print("\n=== GEMINI VIDEO ANALYSIS OUTPUT ===\n")
# print(output)

In [ ]:
# # New Prompt Text Output using Video URL
# video_url = "https://resonate-media-cdn.internal.linqia.com/influencer-content-converted-videos/2378c553fc1a1edab4ae5c3786903c92c5f4b8d9662f0e5e4e695a6c26c114a1.mp4"
# output = analyze_video_url(video_url, NEW_PROMPT_TEXT)
# print("\n=== GEMINI VIDEO ANALYSIS OUTPUT ===\n")
# print(output)